In [3]:
import os,glob
import sentencepiece_model_pb2 as model


In [ ]:
from transformers import PegasusTokenizer

tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")
tokenizer.save_pretrained("./PEGASUS")

In [ ]:
#replace Dataset with the name of the dataset you are using
#replace /path/to/allVocabFilesforDataset/ with the path to the directory containing all the vocab files for the dataset

os.mkdir('./Dataset-PEGASUS')

for fname in glob.glob('/path/to/allVocabFilesforDataset/*'):
    m = model.ModelProto()
    m.ParseFromString(open("./PEGASUS/spiece.model", "rb").read())
    dir_name = f'./Dataset-PEGASUS/{fname.split("/")[-1][:-4]}'
    
    print(dir_name)
    if not os.path.exists(dir_name): os.mkdir(dir_name)
    
    special_tokens = open(fname, "r").read().split("\n")[:-1]
    for idx,row in enumerate(special_tokens):
        token, score = row.split()
        new_token = model.ModelProto().SentencePiece()
        new_token.piece = token
        new_token.score = float(score)
        m.pieces.append(new_token)

    with open(f'{dir_name}/spiece.model', 'wb') as f:
        f.write(m.SerializeToString())

In [ ]:
import pandas as pd
#replace dataset with the name of the dataset you are using
df = pd.read_csv('/path/to/TokenDistCSVinGenerateCandidateWordsforDataser/')
terms_EBM = df[df['TokenFrom']=='dataset']['Token'].to_list()
freq_ebm = df[df['TokenFrom']=='dataset']['Frequency'].to_list()
split_bart = df[df['TokenFrom'] == 'dataset']['SplitSize'].to_list()

sum_num = 0.
sum_den = 0.
for idx,term in enumerate(terms_EBM):
    sum_num += split_bart[idx]*freq_ebm[idx]
    sum_den += freq_ebm[idx]

print(sum_num/sum_den)


In [ ]:

from transformers import PegasusTokenizer
import glob
from collections import defaultdict

#Replace Dataset with the name of the dataset you are using
dict_scores = defaultdict(lambda : defaultdict(dict))
for fname in sorted(glob.glob(f'./Dataset-PEGASUS/*'),key = lambda x: [int(x.split('/')[-1].split('_')[-3][:-1]),float(x.split('/')[-1].split('_')[-2])]):
    domain_tok = PegasusTokenizer.from_pretrained(fname)
    sum_num = 0.
    sum_den = 0.
    
    for idx,term in enumerate(terms_EBM):
        sum_num += min(len(domain_tok.tokenize(term)),len(domain_tok.tokenize(' '+term)))*freq_ebm[idx]
        sum_den += freq_ebm[idx]

    print(fname, sum_num/sum_den, len(domain_tok))
    key = fname.split('/')[-1].split('_')
    dict_scores[key[-3]][key[-2]] = [round(sum_num/sum_den,2),len(domain_tok)]

for k1 in dict_scores:
    if k1 == '0K': continue
    print('data',end='\t')
    for k2 in dict_scores[k1]:
        print(k2,end='\t')
    print()
    break

for k1 in dict_scores:
    print(k1,end='\t')
    for k2 in dict_scores[k1]:
        print(f'{dict_scores[k1][k2][0]}/{dict_scores[k1][k2][1]}',end='\t')
    print()

In [ ]:
ProbSum[2.56] -- 15K-0.25
da  0.0	        0.25	    0.5	        0.75	    1.0	        2.0	        3.0	        4.0	        5.0	        6.0	        7.0	        8.0	        9.0	        10.0	
0K	2.46/96167	
5K	2.55/96105	2.55/96105	2.55/96106	2.55/96106	2.55/96107	2.55/96109	2.55/96111	2.55/96113	2.55/96115	2.55/96117	2.55/96119	2.55/96121	2.55/96123	2.55/96125	
10K	2.53/96112	2.53/96114	2.53/96116	2.53/96118	2.53/96121	2.53/96130	2.53/96139	2.53/96148	2.53/96157	2.53/96166	2.53/96175	2.53/96184	2.53/96193	2.53/96202	
15K	2.52/96116	2.52/96119	2.52/96122	2.52/96125	2.52/96129	2.52/96142	2.52/96155	2.52/96168	2.52/96181	2.52/96194	2.52/96207	2.52/96220	2.52/96233	2.52/96246	
20K	2.51/96121	2.51/96125	2.51/96130	2.51/96134	2.51/96139	2.51/96157	2.51/96175	2.51/96193	2.51/96211	2.51/96229	2.51/96247	2.51/96265	2.5/96283	2.5/96301	
25K	2.51/96123	2.51/96128	2.51/96133	2.51/96138	2.51/96143	2.51/96163	2.51/96183	2.51/96203	2.5/96223	2.5/96243	2.5/96263	2.5/96283	2.5/96303	2.5/96323	
30K	2.51/96123	2.51/96128	2.51/96133	2.51/96138	2.51/96143	2.51/96163	2.51/96183	2.51/96203	2.5/96223	2.5/96243	2.5/96263	2.5/96283	2.5/96303	2.5/96323	
35K	2.51/96125	2.51/96130	2.51/96136	2.51/96141	2.51/96147	2.51/96169	2.51/96191	2.51/96213	2.5/96235	2.5/96257	2.5/96279	2.5/96301	2.5/96323	2.49/96345	
40K	2.51/96127	2.51/96133	2.51/96139	2.51/96145	2.51/96151	2.51/96175	2.51/96199	2.5/96223	2.5/96247	2.5/96271	2.5/96295	2.5/96319	2.49/96343	2.49/96367	
45K	2.49/96130	2.49/96136	2.49/96143	2.49/96150	2.49/96157	2.49/96184	2.49/96211	2.49/96238	2.49/96265	2.49/96292	2.48/96319	2.48/96346	2.48/96373	2.48/96400	
50K	2.49/96130	2.49/96136	2.49/96143	2.49/96150	2.49/96157	2.49/96184	2.49/96211	2.49/96238	2.49/96265	2.49/96292	2.48/96319	2.48/96346	2.48/96373	2.48/96400	
55K	2.49/96132	2.49/96139	2.49/96146	2.49/96153	2.49/96161	2.49/96190	2.49/96219	2.49/96248	2.49/96277	2.48/96306	2.48/96335	2.48/96364	2.48/96393	2.48/96422	
60K	2.49/96133	2.49/96140	2.49/96148	2.49/96155	2.49/96163	2.49/96193	2.49/96223	2.49/96253	2.49/96283	2.48/96313	2.48/96343	2.48/96373	2.48/96403	2.48/96433	


MIMIC [2.98] -- 20K-6
da	0.0	        0.25	    0.5	        0.75	    1.0	        2.0	        3.0	        4.0	        5.0	        6.0	        7.0	        8.0	        9.0	        10.0	
0K	2.27/96491	
5K	2.61/96151	2.61/96163	2.61/96175	2.61/96187	2.61/96199	2.6/96247	2.6/96295	2.6/96343	2.6/96391	2.6/96439	2.6/96487	2.59/96535	2.59/96583	2.59/96631	
10K	2.59/96177	2.59/96195	2.59/96214	2.59/96232	2.59/96251	2.58/96325	2.58/96399	2.58/96473	2.57/96547	2.57/96621	2.57/96695	2.57/96769	2.38/96843	2.37/96917	
15K	2.58/96194	2.58/96216	2.58/96239	2.58/96262	2.57/96285	2.57/96376	2.57/96467	2.57/96558	2.56/96649	2.56/96740	2.37/96831	2.37/96922	2.37/97013	2.36/97104	
20K	2.45/96212	2.45/96239	2.45/96266	2.45/96293	2.45/96321	2.44/96430	2.44/96539	2.44/96648	2.44/96757	2.25/96866	2.24/96975	2.24/97084	2.24/97193	2.24/97302	
25K	2.45/96221	2.45/96250	2.45/96280	2.44/96309	2.44/96339	2.44/96457	2.44/96575	2.44/96693	2.44/96811	2.24/96929	2.24/97047	2.24/97165	2.24/97283	2.23/97401	
30K	2.46/96232	2.45/96264	2.45/96296	2.45/96328	2.45/96361	2.44/96490	2.44/96619	2.44/96748	2.25/96877	2.24/97006	2.24/97135	2.24/97264	2.24/97393	2.24/97522	
35K	2.46/96241	2.46/96275	2.46/96310	2.45/96344	2.45/96379	2.45/96517	2.45/96655	2.44/96793	2.25/96931	2.25/97069	2.25/97207	2.24/97345	2.24/97483	2.24/97621	
40K	2.46/96250	2.46/96286	2.46/96323	2.45/96360	2.45/96397	2.45/96544	2.44/96691	2.44/96838	2.25/96985	2.25/97132	2.25/97279	2.24/97426	2.24/97573	2.23/97720	
45K	2.45/96259	2.45/96298	2.45/96337	2.45/96376	2.45/96415	2.44/96571	2.44/96727	2.3/96883	2.24/97039	2.24/97195	2.24/97351	2.24/97507	2.23/97663	2.23/97819	
50K	2.45/96263	2.45/96303	2.45/96343	2.45/96383	2.44/96423	2.44/96583	2.44/96743	2.25/96903	2.24/97063	2.24/97223	2.24/97383	2.24/97543	2.23/97703	2.23/97863	
55K	2.45/96270	2.45/96311	2.45/96353	2.44/96395	2.44/96437	2.44/96604	2.44/96771	2.24/96938	2.24/97105	2.24/97272	2.24/97439	2.24/97606	2.23/97773	2.23/97940	
60K	2.45/96278	2.45/96321	2.45/96365	2.44/96409	2.44/96453	2.44/96628	2.43/96803	2.24/96978	2.24/97153	2.24/97328	2.23/97503	2.23/97678	2.23/97853	2.23/98028	